### 通过xbox手柄控制 Franka Panda 机器人

* 环境

| Package   | Version |
|-----------|---------|
| gymnasium | No Need  |
| numpy | 1.24.0 |
| grpcio |   1.64.1 |
| pygame | 2.6.0 |
| 关卡名 | Franka_Joystick (可控制关卡，录制或者不录制状态) |
| 关卡名 | Franka_Joystick_Replay （回放专用关卡，不可操作） |

* 通过手柄控制抓夹位姿
* 通过mocap引导机器人移动
* 通过设置录制回放状态，实现控制量的存储和重放，用于示教与模仿学习训练

#### 测试手柄输入

* 运行一段测试程序确认手柄有信号输入
* 按下对应的键会打印信息

In [ ]:
import os
import sys
import time

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

# 将项目根目录添加到 PYTHONPATH
if project_root not in sys.path:
    sys.path.append(project_root)

from orca_gym.devices.xbox_joystick import XboxJoystick

def test_xbox_controller():
    try:
        controller = XboxJoystick()
        while True:
            controller.update()            
            state = controller.get_state()
            print("Buttons:", state["buttons"])
            print("Axes:", state["axes"])
            print("Hats:", state["hats"])
            time.sleep(1)
    except KeyboardInterrupt:
        controller.close()
        print("程序结束")

if __name__ == '__main__':
    test_xbox_controller()


#### 控制 Franka Panda 机械臂

* 基于Panda Mocap 环境，但不处理强化学习相关数据
* 基于手柄状态对mocap点输入偏移量

In [2]:
import os
import sys

current_file_path = os.path.abspath('')
project_root = os.path.dirname(current_file_path)

# 将项目根目录添加到 PYTHONPATH
if project_root not in sys.path:
    sys.path.append(project_root)


import gymnasium as gym
import asyncio
import nest_asyncio
from gymnasium.envs.registration import register
from datetime import datetime
from envs.orca_gym_env import ActionSpaceType
from envs.franka_control.franka_joystick_env import RecordState


nest_asyncio.apply()

# TIME_STEP = 0.016666666666666
TIME_STEP = 0.005

def register_env(grpc_address, record_state, record_file):
    print("register_env: ", grpc_address)
    gym.register(
        id=f"XboxControl-v0-OrcaGym-{grpc_address[-2:]}",
        entry_point="envs.franka_control.franka_joystick_env:FrankaJoystickEnv",
        kwargs={'frame_skip': 1,   # 1 action per frame
                'reward_type': "dense",
                'action_space_type': ActionSpaceType.CONTINUOUS,
                'action_step_count': 0,
                'grpc_address': grpc_address, 
                'agent_names': ['Panda'], 
                'time_step': TIME_STEP,
                'record_state': record_state,
                'record_file': record_file},
        max_episode_steps=60 * 60 * 60,  # 60fps @ 1 hour
        reward_threshold=0.0,
    )

async def continue_training(env):
    observation, info = env.reset(seed=42)
    while True:
        start_time = datetime.now()

        action = env.action_space.sample()
        observation, reward, terminated, truncated, info = env.step(action)

        # 帧率为 60fps ，为显示为正常速度，每次渲染间隔 16ms
        elapsed_time = datetime.now() - start_time
        if elapsed_time.total_seconds() < TIME_STEP:
            await asyncio.sleep(TIME_STEP - elapsed_time.total_seconds())

    

if __name__ == "__main__":
    try:
        grpc_address = "localhost:50051"
        print("simulation running... , grpc_address: ", grpc_address)
        env_id = f"XboxControl-v0-OrcaGym-{grpc_address[-2:]}"

        # RecordState 控制录制和回放状态
        register_env(grpc_address, RecordState.NONE, 'xbox_control_record.h5')

        env = gym.make(env_id)        
        print("启动仿真环境")

        asyncio.run(continue_training(env))
    except KeyboardInterrupt:
        print("关闭仿真环境")        
        env.save_record()
        env.close()

simulation running... , grpc_address:  localhost:50051
register_env:  localhost:50051
Initializing simulation: Class: OrcaGymEnv
Opt config:  {'timestep': 0.004999999888241291, 'apirate': 100.0, 'impratio': 1.0, 'tolerance': 9.99999993922529e-09, 'ls_tolerance': 0.009999999776482582, 'noslip_tolerance': 9.999999974752427e-07, 'mpr_tolerance': 9.999999974752427e-07, 'gravity': [0.0, 0.0, -9.8100004196167], 'wind': [0.0, 0.0, 0.0], 'magnetic': [0.0, -0.5, 0.0], 'density': 1.225000023841858, 'viscosity': 1.8000000636675395e-05, 'o_margin': 0.0, 'o_solref': [0.019999999552965164, 1.0], 'o_solimp': [0.8999999761581421, 0.949999988079071, 0.0010000000474974513, 0.5, 2.0], 'o_friction': [1.0, 1.0, 0.004999999888241291, 9.999999747378752e-05, 9.999999747378752e-05], 'integrator': 3, 'cone': 0, 'jacobian': 2, 'solver': 2, 'iterations': 100, 'ls_iterations': 50, 'noslip_iterations': 100, 'mpr_iterations': 50, 'disableflags': 0, 'enableflags': 0, 'disableactuator': 0, 'sdf_initpoints': 40, 'sdf_i

c:\Users\super\anaconda3\envs\orca_gym\Lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `reset()` method was expecting numpy array dtype to be float64, actual type: int32
  logger.warn(
c:\Users\super\anaconda3\envs\orca_gym\Lib\site-packages\gymnasium\utils\passive_env_checker.py:135: UserWarning: WARN: The obs returned by the `step()` method was expecting numpy array dtype to be float64, actual type: int32
  logger.warn(


关闭仿真环境


c:\Users\super\anaconda3\envs\orca_gym\Lib\site-packages\gymnasium\core.py:311: UserWarning: WARN: env.save_record to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.save_record` for environment variables or `env.get_wrapper_attr('save_record')` that will search the reminding wrappers.
  logger.warn(
